In [ ]:
'''
在alexnet网络下加上数据增强
1.先得到训练集和测试集的图片地址
2.将测试集的图片扩展后保存(736)
3.将训练集的图片提取特征后保存（2934）
'''

In [1]:
'''
运行条件：alex_net权值文件、
          处理好的图片文件(image_file_name,pkl文件，里面是字典)
          image_dict{
                          'train':{'image':{....},'label':{}},
                        'test':{'image':{....},'label':{}},
                    }一般来说存储的时候类型为uint8,提取的时候再转为float32,这样节省空间
'''
#程序的一些参数设定
#设定可以训练与不可训练的网络层
#               conv1     conv2      conv3       conv4     conv5    fc6   fc7     fc8
variable_trable=[False,    False,     False,    False,    False,  False,  True,  True,]
#处理好的图片文件
image_file_name='flower.pkl'
#权值文件
MODEL_ADDR=r'F:\CLASSIC_MODEL\ALEXNET\bvlc_alexnet.npy'
#Tensorboard保存的文件名
log_file_name='fc7tofc8_DataAugment_4long'
#选择是否保存提取出来的特征值
save_feature=True

In [2]:
import numpy as np
#确保下载的权值文件和这个ipython文件再同一个文件夹里面，或者自己指定绝对路径
import tensorflow as tf
from scipy.misc import imread
from scipy.misc import imresize
#from class_name import class_names
import time
import os
import pickle

#                 conv1               conv2           conv3           conv4             conv5
input_shape_list=[[None,227,227,3],[None,27,27,96],[None,13,13,256],[None,13,13,384],[None,13,13,384],\
                 # fc6            fc7           fc8
                  [None,9216],[None,4096],[None,4096]\
                 ]
for vindex,vi in enumerate(variable_trable):
    if vi:
        input_shape__=input_shape_list[vindex]
        layer_index=vindex
        break

variable_trable=[None]+variable_trable

In [3]:
def conv(input, kernel, biases, c_o, s_h, s_w, padding="VALID", group=1):
    '''From https://github.com/ethereon/caffe-tensorflow
    '''
    c_i = input.get_shape()[-1]
    convolve = lambda i, k: tf.nn.conv2d(i, k, [1, s_h, s_w, 1], padding=padding)
    if group == 1:
        #不对输入分组卷积
        conv = convolve(input, kernel)
    else:
        #将输入平分成group组，按[N,w,h,channel]->[0,1,2,3]也就是按输入的channel来分成两个矩阵
        input_groups = tf.split(input, group, 3)  # tf.split(3, group, input)
        #将卷积核平分成group组，按[w,h,in_channel,out_channel]->[0,1,2,3]也就是按输入的channel来分成两个矩阵
        kernel_groups = tf.split(kernel, group, 3)  # tf.split(3, group, kernel)
        #分组卷积
        output_groups = [convolve(i, k) for i, k in zip(input_groups, kernel_groups)]
        #连接卷积的结果
        conv = tf.concat(output_groups, 3)  # tf.concat(3, output_groups)
    return conv + biases

def lrn(x):
    #return x
    #lrn层，现在比较少用，一般用bn层代替
    return tf.nn.local_response_normalization(x,
                                              depth_radius=2,
                                              alpha=2e-05,
                                              beta=0.75,
                                              bias=1.0)
def maxpool(x):
    #因为alex net 用到的maxpool都是一样的参数，所以直接写以函数代替，不用填参数
    return tf.nn.max_pool(x, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='VALID')

def load_model_weight_and_biases():
    '''
    读取模型中的变量值，返回训练好的权重
    model_addr：模型的路径
    '''
    weights_dict = np.load(MODEL_ADDR, encoding='bytes').item()
    return weights_dict

In [4]:
def alexnet(x,net_data,keep_prob,bn_training=True,train=True,extrater_layer=8):#提取特征的时候train=False
    '''
    train=False:特征提取模式，会跑整个model
    train=True:训练模式，只跑需要训练的层
    '''
    input_flag=train
    op_list=[]
    #layer_1 conv1-relu-lrn-maxpool
    if variable_trable[1] or (not train):# 0 or 1
        with tf.name_scope('layer_1'):
            CONV1_W,CONV1_b=tf.Variable(net_data['conv1'][0],name='conv1_w',trainable=variable_trable[1]),\
            tf.Variable(net_data['conv1'][1],name='conv1_b',trainable=variable_trable[1])
            conv1_=conv(X, CONV1_W, CONV1_b, c_o=96, s_h=4, s_w=4, padding="VALID", group=1)
            relu1_=tf.nn.relu(conv1_)
            norm1=lrn(relu1_)#55*55*96
            maxpool1_=maxpool(norm1)#27*27*96
            if not train:op_list.append(maxpool1_)
    #layer_2 conv2-relu-lrn-maxpool
    if variable_trable[2] or (not train):
        if input_flag:
            maxpool1_=x
            input_flag=False
        with tf.name_scope('layer_2'):
            CONV2_W,CONV2_b=tf.Variable(net_data['conv2'][0],name='conv2_w',trainable=variable_trable[2]), \
            tf.Variable(net_data['conv2'][1],name='conv2_b',trainable=variable_trable[2])
            conv2_=conv(maxpool1_, CONV2_W, CONV2_b, c_o=256, s_h=1, s_w=1, padding="SAME", group=2)#27*27*256
            relu2_=tf.nn.relu(conv2_)
            norm2=lrn(relu2_)
            maxpool2_=maxpool(norm2)
            if not train:op_list.append(maxpool2_)#13*13*256
        
    #layer_3 conv3-relu
    if variable_trable[3] or (not train):
        if input_flag:
            maxpool2_=x
            input_flag=False
        with tf.name_scope('layer_3'):
            CONV3_W,CONV3_b=tf.Variable(net_data['conv3'][0],name='conv3_w',trainable=variable_trable[3]),\
            tf.Variable(net_data['conv3'][1],name='conv3_b',trainable=variable_trable[3])
            conv3_=conv(maxpool2_, CONV3_W, CONV3_b, c_o=384, s_h=1, s_w=1, padding="SAME", group=1)#13*13*384
            relu3_=tf.nn.relu(conv3_)
            if not train:op_list.append(relu3_)#13*13*384
    #layer_4 conv4-relu
    if variable_trable[4] or (not train):
        if input_flag:
            relu3_=x
            input_flag=False
        with tf.name_scope('layer_4'):
            CONV4_W,CONV4_b=tf.Variable(net_data['conv4'][0],name='conv4_w',trainable=variable_trable[4]), \
            tf.Variable(net_data['conv4'][1],name='conv4_b',trainable=variable_trable[4])
            conv4_=conv(relu3_, CONV4_W, CONV4_b, c_o=384, s_h=1, s_w=1, padding="SAME", group=2)#13*13*384
            relu4_=tf.nn.relu(conv4_)
            if not train:op_list.append(relu4_)#13*13*384
    
    #layer_5 conv5-relu-maxpool
    if variable_trable[5] or (not train):
        if input_flag:
            relu4_=x
            input_flag=False
        with tf.name_scope('layer_5'):
            CONV5_W,CONV5_b=tf.Variable(net_data['conv5'][0],name='conv5_w',trainable=variable_trable[5]), \
            tf.Variable(net_data['conv5'][1],name='conv5_b',trainable=variable_trable[5])
            conv5_=conv(relu4_, CONV5_W, CONV5_b, c_o=256, s_h=1, s_w=1, padding="SAME", group=2)
            relu5_=tf.nn.relu(conv5_)#13*13*256
            maxpool5_=maxpool(relu5_)
            if not train:op_list.append(maxpool5_)#6*6*256
    if variable_trable[6] or (not train):
        if input_flag:
            maxpool5_=x
            input_flag=False
        with tf.name_scope('layer_6'):
            floatten_input=tf.reshape(maxpool5_,[-1,9216])#N*9216
            floatten_input=tf.nn.dropout(x=floatten_input,keep_prob=keep_prob)
            fc6_w,fc6_b=tf.Variable(net_data['fc6'][0],name='fc6_w',trainable=variable_trable[6]), \
            tf.Variable(net_data['fc6'][1],name='fc7_b',trainable=variable_trable[6])
            fc6_=tf.matmul(floatten_input,fc6_w)+fc6_b
            relu6_=tf.nn.relu(fc6_)#N*4096
            if not train:op_list.append(relu6_)
    if variable_trable[7] or (not train):
        if input_flag:
            relu6_=x
            input_flag=False
        with tf.name_scope('layer_7'):
            relu6_=tf.nn.dropout(x=relu6_,keep_prob=keep_prob)
            fc7_w,fc7_b=tf.Variable(net_data['fc7'][0],name='fc7_w',trainable=variable_trable[7]),\
            tf.Variable(net_data['fc7'][1],name='fc7_b',trainable=variable_trable[7])
            fc7_=tf.matmul(relu6_,fc7_w)+fc7_b
            tf.layers.batch_normalization(fc7_,training=bn_training)
            relu7_=tf.nn.relu(fc7_)#N*4096
            #加入tensor的直方图监视
            tf.summary.histogram('fc7_W',fc7_w)
            tf.summary.histogram('fc7_b',fc7_b)
            tf.summary.histogram('fc7_Wx_plus_b',fc7_)
            tf.summary.histogram('fc7_activate',relu7_)
            if not train:op_list.append(relu7_)
            
    if variable_trable[8] or (not train):
        if input_flag:
            relu7_=x
            input_flag=False
        with tf.name_scope('layer_8'):
    #         fc8_w,fc8_b=tf.Variable(net_data['fc8'][0],name='fc8_w',trainable=variable_trable[8]), \
    #         tf.Variable(net_data['fc8'][1],name='fc8_b',trainable=variable_trable[8])
            relu7_=tf.nn.dropout(x=relu7_,keep_prob=keep_prob)
            #最后一层fc层必须要重新训练
            fc8_w=tf.Variable(tf.truncated_normal(shape=[4096,5],stddev=0.01),dtype=tf.float32,name='fc8_w',\
                              trainable=variable_trable[8])
            fc8_b=tf.Variable(tf.zeros(shape=[5]),dtype=tf.float32,name='fc8_b',trainable=variable_trable[8])
            fc8_=tf.matmul(relu7_,fc8_w)+fc8_b#N*1000
            #添加权值的tensorboard监视
            tf.summary.histogram('fc8_W',fc8_w)
            tf.summary.histogram('fc8_b',fc8_b)
            
            if not train:op_list.append(fc8_)
    #     return fc8_
    return fc8_ if train else op_list[extrater_layer-1]
    
    

In [5]:
reg_pen=tf.reduce_sum([tf.reduce_sum(tf.square(i)) for i in tf.trainable_variables()])

In [6]:
netdata=load_model_weight_and_biases()
with tf.name_scope('input'):
    X=tf.placeholder(dtype=tf.float32,shape=[None,227,227,3])
    Y=tf.placeholder(dtype=tf.float32,shape=[None,5])
    FX=tf.placeholder(dtype=tf.float32,shape=input_shape__)
    KEEP_PROB=tf.placeholder(dtype=tf.float32)
    LEARNRATE=tf.placeholder(dtype=tf.float32)
    BN_TRANING=tf.placeholder(tf.bool)#BN占位符，训练标志位
with tf.name_scope('predict'):
    y_pre=alexnet(FX,netdata,KEEP_PROB)
    #prob=tf.nn.softmax(y_pre)
    #修改测试集，对于一张测试集扩展成10张，计算平均得分,测试集取32张图片
    #转换成batch size的平均分
    mean_score=tf.reduce_mean(tf.reshape(y_pre,[-1,10,5]),axis=1)
    mean_lable=tf.reduce_mean(tf.reshape(Y,[-1,10,5]),axis=1)
    acc_c_t=tf.equal(tf.arg_max(mean_score,1),tf.arg_max(mean_lable,1))
    accuracy_t=tf.reduce_mean(tf.cast(x=acc_c_t,dtype=tf.float32))#测试集准确率，loss
    loss_test=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=mean_score,labels=mean_lable))
#在不需要抽取特征的时候注释extrator网络，因为 extrator和predict是两组不一样的网络，这样能节省显存
# with tf.name_scope('extrator'):
#     feature=alexnet(X,netdata,KEEP_PROB,train=False,extrater_layer=layer_index)
#loss
with tf.name_scope('loss'):
    loss=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_pre,labels=Y))
    loss+=7e-3*reg_pen
with tf.name_scope('trainer'):
    trainer=tf.train.AdamOptimizer(1e-3).minimize(loss)
with tf.name_scope('accuracy'):
    acc_c=tf.equal(tf.arg_max(y_pre,1),tf.arg_max(Y,1))
    accuracy=tf.reduce_mean(tf.cast(x=acc_c,dtype=tf.float32))
sess=tf.InteractiveSession()
log_root=r'D:\Data warehouse\temp_dump\mylog'
writer=tf.summary.FileWriter(log_root)
init=tf.global_variables_initializer()
log_file_name='train_'+log_file_name
writer=tf.summary.FileWriter(os.path.join(log_root,log_file_name))
log_file_name='test_'+log_file_name[6:]
writer_=tf.summary.FileWriter(os.path.join(log_root,log_file_name))
tf.summary.scalar('loss',loss)
tf.summary.scalar('accuracy',accuracy)
merge=tf.summary.merge_all()

In [ ]:
#下面这部分为制作训练集，测试集，如果已存在不用运行这部分程序

In [ ]:
/**************************************************************************************/

In [ ]:
sess.run(init)

In [ ]:
'''
下面将训练集图片转换成特征：
1.得到一张训练图片
2.扩展200倍
3.送入alexnet网络提取特征
4.保存
'''

In [ ]:
import FlowerData as fd
import numpy as np
import os
import data_augmentation as da
import pickle

In [ ]:

img_file_root=r'D:\Data warehouse\5 flower\flower_photos'
#先用图片地址来划分训练集，测试集
img_addr_dict=fd.GetFlowerPicAddr(img_file_root)
fp=open(r'D:\Data warehouse\temp_dump\flower_dict.pkl','wb')
#把这个划分好的地址存起来，后面多次会用到
#（因为划分的mask是随机的，所以为了避免测试与训练集合重复，必须用同一个地址字典）
pickle.dump(file=fp,obj=img_addr_dict)
fp.close()

In [ ]:
fp=open(r'D:\Data warehouse\temp_dump\flower_dict.pkl','rb')
img_addr_dict=pickle.load(fp)
fp.close()
feature_mat,label_mat=[],[]
picture_counter=0
counter=0
batch_num=0

In [ ]:
for i in fd.YieldTrainImage(img_addr_dict):#yield一次一次的加载，避免全部加载给内存太大压力
    x_,y_=map(np.asarray,i)
    TR_FEA=sess.run(feature,feed_dict={X:x_,Y:y_,KEEP_PROB:1.})#转换成特征
    TR_FEA=TR_FEA.reshape((200,-1))
    feature_mat.append(TR_FEA)
    label_mat.append(y_)
    counter+=1
    picture_counter+=1
    if counter%100==0:
        print('batch counter:{},picturenum:{}/2934,batch num:{}'.format(counter,picture_counter,batch_num))
    #按batch保存,每个batch保存 200*400=80000条特征，最后一个batch大概20000条
    if counter>80*5:
        counter=0
        batch_num+=1
        fea_filename='feature_batch_'+str(batch_num)+'.pkl'
        froot=r'D:\Data warehouse\temp_dump\feature_batch'
        feature_mat,label_mat=map(np.concatenate,[feature_mat,label_mat])
        fp=open(os.path.join(froot,fea_filename),'wb')
        idict={'feature':feature_mat.astype(np.float32),'lable':label_mat.astype('uint8')}
        pickle.dump(file=fp,obj=idict)
        fp.close()
        del idict
        feature_mat,label_mat=[],[]
        print('---batch num:{} save,picturenum:{}/2934'.format(batch_num,picture_counter))
#最后一个batch
if not counter == 0:
    batch_num+=1
    fea_filename='feature_batch_'+str(batch_num)+'.pkl'
    froot=r'D:\Data warehouse\temp_dump\feature_batch'
    feature_mat,label_mat=map(np.concatenate,[feature_mat,label_mat])
    fp=open(os.path.join(froot,fea_filename),'wb')
    idict={'feature':feature_mat.astype(np.float16),'lable':label_mat.astype('uint8')}
    pickle.dump(file=fp,obj=idict)
    fp.close()
    del idict
    feature_mat,label_mat=[],[]
    print('---batch num:{} save,picturenum:{}/2934'.format(batch_num,picture_counter))


In [ ]:
'''
下面制作测试集
'''
#读取地址
fp=open(r'D:\Data warehouse\temp_dump\flower_dict.pkl','rb')
img_dict=pickle.load(fp)
fp.close()
#保存扩展后的测试图片
fd.SaveTestImage(img_dict)

In [ ]:
#抽取测试集特征
fp=open(r'D:\Data warehouse\temp_dump\flower_test_img.pkl','rb')
test_imgdict=pickle.load(fp)
fp.close()
test_feature={'feature':None,'lables':None}
test_feature_mat,test_feature_lable=[],[]
print('converting test image to feature!')
#
test_feature_mat=[]
test_feature_lable=[]
for fname in test_imgdict:#按种类，每10张（由一张原图扩展得来）送入网络抽取特征
    x_all=test_imgdict[fname]['image'].astype(np.float32)
    y_all=test_imgdict[fname]['lables'].astype(np.float32)
    pic_num=x_all.shape[0]
    print(fname,pic_num)
    #一张图扩展成10张，每10张输入网络提取
    loop_num=pic_num//10
    s_index=0
    for l_ in range(loop_num):
        TR_FEA=sess.run(feature,feed_dict={X:x_all[s_index:s_index+10],Y:y_all[s_index:s_index+10],KEEP_PROB:1.})
        TR_FEA=TR_FEA.reshape((1,10,-1))#第一维度仍然保持为 736，方便取样
        test_feature_mat.append(TR_FEA)
        s_index+=10
        test_feature_lable.append(test_imgdict[fname]['lables'][:10].reshape((1,10,5)))
test_feature_mat,test_feature_lable=map(np.concatenate,[test_feature_mat,test_feature_lable])
test_feature_dict={'feature':test_feature_mat,'lable':test_feature_lable}
fp=open(r'D:\Data warehouse\temp_dump\feature_batch\test_feature.pkl','wb')
pickle.dump(file=fp,obj=test_feature_dict)
fp.close()
print('save test feature!')

In [ ]:
/*************************************************************************/

In [7]:
sess.run(init)

In [8]:
fp=open(r'D:\Data warehouse\temp_dump\feature_batch\test_feature.pkl','rb')
test_fea=pickle.load(fp)
fp.close()

In [9]:
test_fea['lable'].shape

(736, 10, 5)

In [10]:
#bn操作符
bn_ops=tf.get_collection(tf.GraphKeys.UPDATE_OPS)

In [ ]:
#……………………………………训练网络……………………………………………………
#%debug
kp_te,kp_tr=1.,0.5
lr=1e-3
run_counter=0
for i in range(8*100):
    print('load new batch!!')
    batch_id=np.random.choice(list(range(1,9)),3,replace=False)
    train_fea={'feature':[],'lable':[]}
    #加载3个batch文件到内存,每个batch有400*200=80000条特征
    for t_id in batch_id:
        fea_filename='feature_batch_'+str(t_id)+'.pkl'
        froot=r'D:\Data warehouse\temp_dump\feature_batch'
        fp=open(os.path.join(froot,fea_filename),'rb')
        t_fea=pickle.load(fp)
        fp.close()
        train_fea['feature'].append(t_fea['feature'])
        train_fea['lable'].append(t_fea['lable'])
    train_fea['feature']=np.concatenate(train_fea['feature'])
    train_fea['lable']=np.concatenate(train_fea['lable'])
    #k*10*10*N
    for k in range(62*5):
        mask=np.random.choice(train_fea['lable'].shape[0],512,replace=False)
        x_,y_=train_fea['feature'][mask].astype(np.float32),train_fea['lable'][mask].astype(np.float32)
        
        for j in range(10):
            run_counter+=1
            sess.run([trainer,bn_ops],feed_dict={FX:x_,Y:y_,KEEP_PROB:kp_tr,LEARNRATE:lr,BN_TRANING:True})
        if run_counter%100==0:
            loss_,acc_,m_=sess.run([loss,accuracy,merge],feed_dict={FX:x_,Y:y_,KEEP_PROB:kp_tr,LEARNRATE:lr})
            writer.add_summary(m_,run_counter)
            print('epoch:{},loss:{},train accuracy:{}'.format(run_counter,loss_,acc_))
        if run_counter%(200*3)==0:
            mask=np.random.choice(test_fea['feature'].shape[0],128,replace=False)
            #32*10*227*227*3,32*5
            x_,y_=(test_fea['feature'][mask]).reshape((1280,-1)),test_fea['lable'][mask].reshape((1280,-1))
            acc_,loss_,m_,_=sess.run([accuracy_t,loss_test,merge,bn_ops],\
                                     feed_dict={FX:x_,Y:y_,KEEP_PROB:1.,LEARNRATE:lr,BN_TRANING:False})
            writer_.add_summary(m_,run_counter)
            print('--epoch:{}/3760000,test loss:{},test accuracy:{}'.format(run_counter,loss_,acc_))
    if i%8==0:
        lr=max(1e-5,0.99*lr)
        print('--epoch:{},New learning rate:{}'.format(run_counter,lr))

In [ ]:
test_fea['feature'][mask].shape

In [ ]:
writer.close()
writer_.close()

In [ ]:
fp=open(r'D:\Data warehouse\temp_dump\feature_batch\test_feature.pkl','rb')
test_fea=pickle.load(fp)

In [ ]:
test_fea['feature'].shape

In [ ]:
c=0
for i in test_fea:
    c+=test_fea['feature'].shape[0]

In [ ]:
c

In [ ]:
#mask=np.random.choice(test_fea['feature'].shape[0],128,replace=False)
x_,y_=test_fea['feature'][100:200],test_fea['lable'][100:200]
acc_=sess.run(mean_lable,feed_dict={FX:x_,Y:y_,KEEP_PROB:1.,LEARNRATE:1e-4})
print(acc_)

In [ ]:
acc_